In [ ]:
!wandb login

In [3]:
# 下载tiny karpathy dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-05-31 05:07:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-05-31 05:07:03 (18.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# 基于sub_word(there is per char) 构建简单的字符集tokrnizer映射
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [35]:
char_set = sorted(list(set(text)))
vocab_size = len(char_set)

In [36]:
char_set

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [37]:
' '.join(char_set)

"\n   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [38]:
vocab_size #可以设置corpus的键为65个

65

In [39]:
""""
simple strategy for tokenize the input
"""
str2num = {ch:i for i,ch in enumerate(char_set)}
num2str = {i:ch for i,ch in enumerate(char_set)}

encode = lambda s: [str2num[ch] for ch in s] #encoder:translate a string to a list of int
decode = lambda l: ''.join([num2str[i] for i in l]) #docoder:translate a list of int to output:string

In [40]:
print(encode("nihao i love u"))
print('\n')
print(decode(encode("nihao i love u")))

[52, 47, 46, 39, 53, 1, 47, 1, 50, 53, 60, 43, 1, 59]


nihao i love u


In [41]:
# 将整个dataset进行tokenize
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1115394]) torch.int64


In [42]:
# 划分90%训练集和10%验证集
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data),len(val_data))

1003854 111540


随机采样一定的数据来输入Transformer中，避免一次性输入

In [43]:
# import wandb
# wandb.init(project="Mini_GPT")
# config = wandb.config
max_size_per_token_input = 8
batch_size = 4
block_size = 8
lr=1e-3
dropout = 0.2
n_embd = 32
n_layer = 4
n_head = 4

In [44]:
max_size_per_token_input = 8
# 进行CBOW，预测末位token
x = train_data[:max_size_per_token_input]
y = train_data[1:max_size_per_token_input+1]
for _ in range(max_size_per_token_input):
  context = x[:_+1]
  target = y[_]
  print(f"input context {context},target {target}")

input context tensor([18]),target 47
input context tensor([18, 47]),target 56
input context tensor([18, 47, 56]),target 57
input context tensor([18, 47, 56, 57]),target 58
input context tensor([18, 47, 56, 57, 58]),target 1
input context tensor([18, 47, 56, 57, 58,  1]),target 15
input context tensor([18, 47, 56, 57, 58,  1, 15]),target 47
input context tensor([18, 47, 56, 57, 58,  1, 15, 47]),target 58


In [45]:
torch.manual_seed(1337)  #设置随机种子，方便复现
batch_size = 4  #并行输入的独立序列，batch容量
block_size = 8  #context最大token

def get_batch(dstype):
  data = train_data if dstype == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,)) #size_template 1dim (batch_size=4,)
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y #每个batch中sequence对应的context和target

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [46]:
for _ in range(batch_size):
  for i in range(block_size):
    context = xb[_,:i+1]
    target = yb[_,i]
    print(f"sequence{_}：input context {context},target {target}")

sequence0：input context tensor([24]),target 43
sequence0：input context tensor([24, 43]),target 58
sequence0：input context tensor([24, 43, 58]),target 5
sequence0：input context tensor([24, 43, 58,  5]),target 57
sequence0：input context tensor([24, 43, 58,  5, 57]),target 1
sequence0：input context tensor([24, 43, 58,  5, 57,  1]),target 46
sequence0：input context tensor([24, 43, 58,  5, 57,  1, 46]),target 43
sequence0：input context tensor([24, 43, 58,  5, 57,  1, 46, 43]),target 39
sequence1：input context tensor([44]),target 53
sequence1：input context tensor([44, 53]),target 56
sequence1：input context tensor([44, 53, 56]),target 1
sequence1：input context tensor([44, 53, 56,  1]),target 58
sequence1：input context tensor([44, 53, 56,  1, 58]),target 46
sequence1：input context tensor([44, 53, 56,  1, 58, 46]),target 39
sequence1：input context tensor([44, 53, 56,  1, 58, 46, 39]),target 58
sequence1：input context tensor([44, 53, 56,  1, 58, 46, 39, 58]),target 1
sequence2：input context tens

In [17]:
# Bi_gram Language Model  (Boliean Embedding)
import torch
from torch.nn import functional as F
import torch.nn as nn

torch.manual_seed(1337)

class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx) #batch_size*block_size*vocab_size
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      logits, loss = self(idx) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样处最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

In [18]:
model = BiGramLM(vocab_size)
logits, loss = model(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [19]:
decode(model.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=250)[0].tolist())

"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca!UQ!vrpxZQgC-hlkq,ptKqHoiX-jjeLJ &slERj KUsBOL!mpJ"

In [47]:
# 创建优化器
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) # 自注意力不适用高的学习率，而学习率降低，则需要提高训练迭代次数，以保持性能。并且模型增大时，也需要降低学习率，来防止局部最优

In [21]:
batch_size = 32
# 验证集，用于在训练时观察验证情况，train_loss > val_loss 说明训练正常,小于时则说明有过拟合的风险，此时观察epoch进行选择
for epoch in range(10000+1):
  xb, yb = get_batch('train')
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
  if epoch % 1000 == 0:
    print(f"epoch {epoch},loss {loss.item()}")

epoch 0,loss 4.711565017700195
epoch 1000,loss 3.7984933853149414
epoch 2000,loss 3.0819954872131348
epoch 3000,loss 2.765151023864746
epoch 4000,loss 2.633697748184204
epoch 5000,loss 2.5281035900115967
epoch 6000,loss 2.6245577335357666
epoch 7000,loss 2.476557493209839
epoch 8000,loss 2.564079999923706
epoch 9000,loss 2.4546542167663574
epoch 10000,loss 2.4646947383880615


In [22]:
res = decode(model.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=1000)[0].tolist())

In [23]:
res

"\nThixqu tes; st yo hind wotte grotonear 'so it t jod weancotha:\nh hay.JUCle n prids, r loncave w hollular s O:\nHIs; ht anjx?\n\nDUThineent.\n\nLavinde.\nathave l.\nKEONGBUCHandspo be y,-hedarwnoddy scace, tridesar, wne'shenous s ls, theresseys\nPlorseelapinghiybHen yof GLUCEN t l-t E:\nI hisgothers w dere! ABer wotouciullle's fldrwertho s?\nNDan'spererds cist ripl chys er orlese;\nYo jehof h hecere ekn wferommot mowo soaf yoit, ince his, t, f at. fal whetrimy bupof tor atha Bu!\nJOutho f cimimave.\nNEDUSt cir selle p wie wede\nRo n apenor f'Y tover witys an sh d w t e w!\nCEOntiretoaveE IINpe, theck. cung.\nORIsthies hacin benqurd bll, d a r w wistatsowor ath\nFivet bloll ang a-I theeancusemee tsce larry t I lag sze t\nLCKI thit,\nn.\nFaure ds ppplirn!\nmeftou ow pring, avewist th;\nTENTEMETCI gienco, An he waro whiougou he s imaror?\nBu ne-ingof acat nd l,\nFothind at wrt:\nHMovint gros!\nAMQUThes med thestw cos wand herf s hafold mZWirus je ney biPos t ngabsestouMOLAUCES: ONDoher

# self-attention module

In [48]:
# 创建单头自注意力
# decoder结构
class OneHead(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(n_embd,head_size,bias=False)
    self.key = nn.Linear(n_embd,head_size,bias=False)
    self.value = nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size))) #利用buffer创建下三角矩阵
    self.dropout = nn.Dropout(dropout) #全局变量中定义
  def forward(self,x):
    B,T,C = x.shape
    q = self.query(x) #(B,T,head_size)
    k = self.key(x) #(B,T,head_size)
    v = self.value(x) #(B,T,head_size)
    # 计算attention score
    weight = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,head_size) @ (B,head_size,T) -> (B,T,T)
    # 除以的根C，故也成缩放点积注意力
    # 这里除以的根号C，是为了将分布归一化，使得输出稳定，而不会出现错峰的峰值
    weight = weight.masked_fill(self.tril[:T,:T] == 0, float('-inf')) #添加掩码，防止未来信息泄露
    # 对于一些不需要隐藏未来信息的任务，可以不用掩码，如情感分析
    weight = F.softmax(weight,dim=-1)
    weight = self.dropout(weight) # 在一定程度上妨碍计算间的通信，dropout来防止过拟合
    out = weight @ v # (B,T,T) @ (B,T,head_size) -> (B,T,head_size)
    return out

In [49]:
torch.manual_seed(1337)

n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # 将cpopus进行嵌入，是对自身信息的处理
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)
    # 对位置关系进行处理
    self.position_embedding_table = nn.Embedding(block_size,n_embd)

  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #batch_size*block_size*n_embd
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #block_size*n_embd
    x = tok_emb + pos_emb #batch_size*block_size*n_embd的2倍
    # x此时包含了自身token信息和位置信息
    logits = self.ln_head(x) #batch_size*block_size*vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:] #限制idx大小，防止位置嵌入表溢出
      logits, loss = self(idx_cond) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)，即序列中的最后一个字词的得分分布
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样处最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

# 多头注意力--并行多个self-attention

In [50]:
class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([OneHead(head_size) for _ in range(num_heads)])  #实例化所包含的多个自注意力头
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout) #全局变量中定义
  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim=-1)
    out = self.proj(out) #残差连接的中间过程，将结果投影到剩余层(路径)
    return out

In [ ]:
torch.manual_seed(1337)

n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # 将cpopus进行嵌入，是对自身信息的处理
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)
    # 对位置关系进行处理
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.multihead = MultiHeadAttention(4,n_embd//4)  # 由于n_embd为32，设置的是4个注意力头，给个注意力头的输入为8-dim

  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #batch_size*block_size*n_embd
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #block_size*n_embd
    x = tok_emb + pos_emb #batch_size*block_size*n_embd
    # x此时包含了自身token信息和位置信息
    x = self.multihead(x)
    logits = self.ln_head(x) #batch_size*block_size*vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:] #限制idx大小，防止位置嵌入表溢出
      logits, loss = self(idx_cond) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)，即序列中的最后一个字词的得分分布
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样处最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

# 前馈层--以单层MLP尝试

In [51]:
class FeedForward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.ReLU(), #ReLU是通用的激活函数，已可以换位LakeReLU或tanh
        nn.Linear(4*n_embd,n_embd), #映射到原来的残差路径
        nn.Dropout(dropout), # 全局变量指定，一般在投影回残差连接路径前进行dropout，来将其作为最后一层
    )

  def forward(self,x):
    return self.net(x)

In [ ]:
torch.manual_seed(1337)

n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # 将cpopus进行嵌入，是对自身信息的处理
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)
    # 对位置关系进行处理
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.multihead = MultiHeadAttention(4,n_embd//4)  # 由于n_embd为32，设置的是4个注意力头，给个注意力头的输入为8-dim
    self.feed_forward = FeedForward(n_embd)

  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #batch_size*block_size*n_embd
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #block_size*n_embd
    x = tok_emb + pos_emb #batch_size*block_size*n_embd
    # x此时包含了自身token信息和位置信息
    x = self.multihead(x)
    x = self.feed_forward(x)
    logits = self.ln_head(x) #batch_size*block_size*vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:] #限制idx大小，防止位置嵌入表溢出
      logits, loss = self(idx_cond) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)，即序列中的最后一个字词的得分分布
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样出最后一个sequence中每个token的最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

# 创建Transformer的block简单结构，主要重复多头注意力

In [52]:
class block(nn.Module):
  """
  这个块结构主要以进行计算间通信的多头注意力构成
  也可以加入加MLP尝试
  """
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.mltHead = MultiHeadAttention(n_head,head_size)
    self.feed_forward = FeedForward(n_embd)
    self.lnm1 = nn.LayerNorm(n_embd)
    self.lnm2 = nn.LayerNorm(n_embd)
  def forward(self,x):
    # x = self.mltHead(x)
    # x = self.feed_forward(x)
    x = x + self.mltHead(self.lnm1(x))
    x = x + self.feed_forward(self.lnm2(x)) #残差连接，防止梯度消失，加快收敛
    return x

In [ ]:
torch.manual_seed(1337)

n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # 将cpopus进行嵌入，是对自身信息的处理
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)
    # 对位置关系进行处理
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    # self.multihead = MultiHeadAttention(4,n_embd//4)  # 由于n_embd为32，设置的是4个注意力头，给个注意力头的输入为8-dim
    # 已经包含在sequential中了
    self.blocks = nn.Sequential(
        block(n_embd,4),
        block(n_embd,4),
        block(n_embd,4),
        block(n_embd,4),
        nn.LayerNorm(n_embd), #到解码的最后一个线性层前再进行一次
    )
    # self.feed_forward = FeedForward(n_embd) #已经包含在sequential中了


  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #batch_size*block_size*n_embd
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #block_size*n_embd
    x = tok_emb + pos_emb #batch_size*block_size*n_embd
    # x此时包含了自身token信息和位置信息
    # x = self.multihead(x)
    # x = self.feed_forward(x)
    x = self.blocks(x)
    logits = self.ln_head(x) #batch_size*block_size*vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:] #限制idx大小，防止位置嵌入表溢出
      logits, loss = self(idx_cond) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)，即序列中的最后一个字词的得分分布
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样出最后一个sequence中每个token的最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

# Transformer中存在的两个优化措施：1、残差连接 2、层间Norm Layer，现在大多是在转化前运用BatchNorm,这与论文不同

In [53]:
torch.manual_seed(1337)

n_embd = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 完善，模型可扩展性，使其结构能重复n次
class BiGramLM(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # 将cpopus进行嵌入，是对自身信息的处理
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)
    # 对位置关系进行处理
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    # self.multihead = MultiHeadAttention(4,n_embd//4)  # 由于n_embd为32，设置的是4个注意力头，给个注意力头的输入为8-dim
    # 已经包含在sequential中了
    self.blocks = nn.Sequential(*[block(n_embd, n_head=n_head) for _ in range(n_layer)]) #全局变量中指定
    self.f_lnm = nn.LayerNorm(n_embd) #最后输出线性层前的最后一次layernorm
    # self.feed_forward = FeedForward(n_embd) #已经包含在sequential中了


  def forward(self,idx,targets=None):
    B,T = idx.shape

    tok_emb = self.token_embedding_table(idx) #batch_size*block_size*n_embd
    pos_emb = self.position_embedding_table(torch.arange(T,device=device)) #block_size*n_embd
    x = tok_emb + pos_emb #batch_size*block_size*n_embd
    # x此时包含了自身token信息和位置信息
    # x = self.multihead(x)
    # x = self.feed_forward(x)
    x = self.blocks(x)
    x = self.f_lnm(x)
    logits = self.ln_head(x) #batch_size*block_size*vocab_size

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self,idx,max_new_tokens):
    """
    idx:(batch_size,block_size)
    max_new_tokens:接下来需要生成的最多token数
    """
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:] #限制idx大小，防止位置嵌入表溢出
      logits, loss = self(idx_cond) #得到预测结果
      logits = logits[:,-1,:]  #需要得到最后的time step，shape(B,C)，即序列中的最后一个字词的得分分布
      probs = F.softmax(logits,dim=-1)  #将分布转换为概率
      idx_next = torch.multinomial(probs,num_samples=1) #采样出最后一个sequence中每个token的最大概率的索引，(B,1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx #shape(B,T+1)

In [54]:
# model = BiGramLM(vocab_size)
# wandb.watch(model,log="all")
for epoch in range(10000+1):
  xb, yb = get_batch('train')
  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
  if epoch % 1000 == 0:
    print(f"epoch {epoch},loss {loss.item()}")

epoch 0,loss 4.380142688751221
epoch 1000,loss 2.7046000957489014
epoch 2000,loss 2.893082857131958
epoch 3000,loss 2.5212631225585938
epoch 4000,loss 2.1198503971099854
epoch 5000,loss 2.2809371948242188
epoch 6000,loss 2.1075241565704346
epoch 7000,loss 2.289177417755127
epoch 8000,loss 1.940026044845581
epoch 9000,loss 2.0970635414123535
epoch 10000,loss 2.111042022705078
